In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [5]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results= 1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
print(wiki)

api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DELL\\Documents\\GitHub\\QA-ChatBot\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)


In [6]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxive = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxive)

api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)


In [7]:
tools = [wiki, arxive]

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings= HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")



USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\DELL\Documents\GitHub\QA-ChatBot\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
splitter =  RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 50)
text_split = splitter.split_documents(docs)
vectordb = FAISS.from_documents(docs, embeddings)
retriever = vectordb.as_retriever()
retriever

NameError: name 'WebBaseLoader' is not defined